# Ateema Capstone Project


## Local models

#### Embedding

[GPT4All Embeddings](https://blog.nomic.ai/posts/nomic-embed-text-v1):

```
pip install langchain-nomic
```

### LLM

Use [Ollama](https://ollama.ai/) and [llama3](https://ollama.ai/library/llama3):

```
ollama pull llama3
```

Prompt -

https://llama.meta.com/docs/model-cards-and-prompt-formats/meta-llama-3/


## Libraries

In [78]:
import pandas as pd
import numpy as np
import os

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.schema import Document
from langgraph.graph import END, StateGraph

from typing_extensions import TypedDict
from typing import List
from pprint import pprint

# LLM
local_llm = 'llama3'

## Setup

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
# data loader
loader = CSVLoader(file_path="/content/drive/MyDrive/stage1_all.csv")
data = loader.load()

In [91]:
from gpt4all import Embed4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# Assume `data` is already loaded

# Data transformers
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
texts = text_splitter.split_documents(data)

# Initialize embeddings
embedder = Embed4All()

# Generate embeddings for documents
embeddings = [embedder.embed(doc.page_content) for doc in texts]

# Create a dictionary to map document content to their embeddings
embedding_dict = {doc.page_content: embedding for doc, embedding in zip(texts, embeddings)}

# Define a custom embedding function
class CustomEmbeddingFunction:
    def __init__(self, embedder, embedding_dict):
        self.embedder = embedder
        self.embedding_dict = embedding_dict

    def embed_documents(self, texts):
        return [self.embedding_dict[text] for text in texts]

    def embed_query(self, text):
        return self.embedder.embed(text)

# Initialize the custom embedding function
custom_embedding = CustomEmbeddingFunction(embedder, embedding_dict)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=texts,
    embedding=custom_embedding,
    collection_name="rag-chroma"
)

# Retriever
retriever = vectorstore.as_retriever()

## Retrieval Grader


In [92]:
# LLM
llm = ChatOllama(model="llama3", format="json", temperature=0)

# Define prompt
prompt = PromptTemplate(
    template="""system You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     user
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n assistant
    """,
    input_variables=["question", "document"],
)

retrieval_grader = prompt | llm | JsonOutputParser()

# Example question
question = "My name is Sanjay. I will be visiting Chicago in summer with my family to explore and would like to go to interesting places."

# Retrieve documents
docs = retriever.get_relevant_documents(question)
doc_txt = docs[0].page_content

# Grade the relevance
result = retrieval_grader.invoke({"question": question, "document": doc_txt})
print(result)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'score': 'yes'}


## Generation


In [93]:
# # Prompt
# prompt = PromptTemplate(
#     template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks.
#     Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
#     Use three sentences maximum and keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
#     Question: {question}
#     Context: {context}
#     Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
#     input_variables=["question", "document"],
# )

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an AI tour guide named Ateema. Your role is to assist users in finding interesting places to visit.
    Greet the user warmly and provide recommendations based on their specified location. Provide a detailed description of each place ranked up to 10 places and why it might be interesting to the user.
    The prompt length targets up to 60 seconds that explicitly states what I asked you to and end a prompt like a professional tour guide as there will be no continuging conversation after generating a prompt. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

llm = ChatOllama(model=local_llm, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = prompt | llm | StrOutputParser()


# Run
question = "My name is Sanjay. I will be visiting Chicago in summer with my family to explore and would like to go to interesting places."

# Understanding retriever relevant metrics
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

Sanjay! Welcome to Chicago in the summer! I'm Ateema, your AI tour guide. I've got some fantastic recommendations for you and your family to enjoy the city's beautiful beaches and more.

As you're looking for interesting places to visit, I'll give you my top 10 picks, ranked from 1 to 10. Here they are:

**Rank 1: North Avenue Beach**
Start your beach-hopping adventure at North Avenue Beach, one of Chicago's most popular spots. Enjoy the sun, sand, and stunning views of Lake Michigan. You can even take a dip in the lake or try your hand at surfing!

**Rank 2: Oak Street Beach**
Next up is Oak Street Beach, a hidden gem with plenty of room to spread out and soak up the sun. Take a stroll along the beachfront path, grab a snack from one of the many food vendors, and enjoy the lively atmosphere.

**Rank 3: Montrose Beach**
Montrose Beach is another must-visit spot for beach lovers. With its picturesque views and calm waters, it's perfect for families or those looking to relax. Don't miss 

## Hallucination Grader


In [94]:
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

## Answer Grader


In [95]:
# LLM
llm = ChatOllama(model=local_llm, format="json", temperature=0)

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)

answer_grader = prompt | llm | JsonOutputParser()
answer_grader.invoke({"question": question,"generation": generation})

{'score': 'yes'}

## Control Flow in LangGraph


In [96]:
### State

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """
    question : str
    generation : str
    documents : List[str]

### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}

def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}


def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)


def decide_to_generate(state):
    """
    Determines whether to generate an answer

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    # We have relevant documents, so generate answer
    print("---DECISION: GENERATE---")
    return "generate"

def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("route_question", route_question) # route question
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generate

## Build Graph

In [97]:
# Build graph
workflow.set_entry_point("route_question")
workflow.add_edge("route_question", "retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "generate": "generate",
    },
)
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "retrieve",
    },
)

## Compile & Test

In [98]:
# Compile
app = workflow.compile()

# Test
inputs = {"question": "My name is Sanjay. I will be visiting Chicago in summer with my family to explore and would like to go to interesting places."}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---ROUTE QUESTION---
My name is Sanjay. I will be visiting Chicago in summer with my family to explore and would like to go to interesting places.
---RETRIEVE---
'Finished running: retrieve:'
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: GENERATE---
'Finished running: grade_documents:'
---GENERATE---
---CHECK HALLUCINATIONS---
---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---
---GRADE GENERATION vs QUESTION---
---DECISION: GENERATION ADDRESSES QUESTION---
'Finished running: generate:'
("Sanjay! Welcome to Chicago in the summer! I'm Ateema, your AI tour guide. "
 "I've got some fantastic recommendations for you and your family to enjoy the "
 "city's beautiful beaches and more.\n"
 '\n'
 "As you're looking for interesting places to visit, I'll give you my top 10 "
 'picks, ranked from 1 to 10. Here they are:\n'
 '\n'
 '**Ra

# SadTalker Avatar video generation + edge_tss text-to-audio-Setup

In [99]:
!pip install numpy scipy pillow opencv-python-headless scikit-image
!pip install gdown paramiko
!pip install kornia
!pip install yacs
!pip install gfpgan
!pip install pydub
!pip install facexlib
!pip install torchaudio==2.0.0
!pip install torch==2.0.0+cu117 torchvision==0.15.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html
!apt-get update
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.9/225.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x

In [100]:
from google.colab import drive
drive.mount('/content/drive')

import os
base_dir = '/content/drive/MyDrive/SadTalker'
checkpoints_dir = os.path.join(base_dir, 'checkpoints')
weights_dir = os.path.join(base_dir, 'weights')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [101]:
# Downloading the checkpoints to Google Drive
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar -O {checkpoints_dir}/mapping_00109-model.pth.tar
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar -O {checkpoints_dir}/mapping_00229-model.pth.tar
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors -O {checkpoints_dir}/SadTalker_V0.0.2_256.safetensors
!gdown https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors -O {checkpoints_dir}/SadTalker_V0.0.2_512.safetensors

# Downloading facexlib and GFPGAN weights to Google Drive
!gdown https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth -O {weights_dir}/alignment_WFLW_4HG.pth
!gdown https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth -O {weights_dir}/detection_Resnet50_Final.pth
!gdown https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth -O {weights_dir}/GFPGANv1.4.pth
!gdown https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth -O {weights_dir}/parsing_parsenet.pth
!gdown https://github.com/Winfredy/SadTalker/releases/download/v0.0.2/epoch_20.pth -O {checkpoints_dir}/epoch_20.pth

# Download the checkpoints to Google Drive if they don't already exist
def download_if_not_exists(url, dest):
    if not os.path.exists(dest):
        !gdown {url} -O {dest}

download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar', f'{checkpoints_dir}/mapping_00109-model.pth.tar')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar', f'{checkpoints_dir}/mapping_00229-model.pth.tar')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors', f'{checkpoints_dir}/SadTalker_V0.0.2_256.safetensors')
download_if_not_exists('https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors', f'{checkpoints_dir}/SadTalker_V0.0.2_512.safetensors')

download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.1.0/alignment_WFLW_4HG.pth', f'{weights_dir}/alignment_WFLW_4HG.pth')
download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth', f'{weights_dir}/detection_Resnet50_Final.pth')
download_if_not_exists('https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth', f'{weights_dir}/GFPGANv1.4.pth')
download_if_not_exists('https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth', f'{weights_dir}/parsing_parsenet.pth')
download_if_not_exists('https://github.com/Winfredy/SadTalker/releases/download/v0.0.2/epoch_20.pth', f'{checkpoints_dir}/epoch_20.pth')

Downloading...
From: https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
To: /content/drive/MyDrive/SadTalker/checkpoints/mapping_00109-model.pth.tar
100% 156M/156M [00:00<00:00, 307MB/s]
Downloading...
From: https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00229-model.pth.tar
To: /content/drive/MyDrive/SadTalker/checkpoints/mapping_00229-model.pth.tar
100% 156M/156M [00:00<00:00, 297MB/s]
Downloading...
From: https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_256.safetensors
To: /content/drive/MyDrive/SadTalker/checkpoints/SadTalker_V0.0.2_256.safetensors
100% 725M/725M [00:02<00:00, 259MB/s]
Downloading...
From: https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/SadTalker_V0.0.2_512.safetensors
To: /content/drive/MyDrive/SadTalker/checkpoints/SadTalker_V0.0.2_512.safetensors
100% 725M/725M [00:04<00:00, 148MB/s]
Error:

	[Errno 2] No such file or directory:
	'/con

In [108]:
!pip install edge-tts

# Text to Audio model TTS

In [113]:
import edge_tts
import asyncio
import nest_asyncio

# Allow nested event loops
nest_asyncio.apply()

# Convert text to audio
def text_to_audio(text, filename):
    async def _convert():
        communicator = edge_tts.Communicate(text, voice="en-US-GuyNeural")
        await communicator.save(filename)

    loop = asyncio.get_event_loop()
    loop.run_until_complete(_convert())

audio_filename = "/content/drive/MyDrive/SadTalker/examples/driven_audio/generated_audio.wav"
text_to_audio(value["generation"], audio_filename)
print(f"Audio file saved as {audio_filename}")

Audio file saved as /content/drive/MyDrive/SadTalker/examples/driven_audio/generated_audio.wav


#Avatar video generation using SadTalker

In [117]:
# SadTalker inference prep
source_image_path = "/content/drive/MyDrive/SadTalker/examples/source_image/obama.png"
result_dir = "/content/drive/MyDrive/SadTalker/results"
checkpoint_dir = "/content/drive/MyDrive/SadTalker/checkpoints"
audio_filename = "/content/drive/MyDrive/SadTalker/examples/driven_audio/generated_audio.wav"

# directories checking
os.makedirs(result_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

#SadTalker inference
!python /content/drive/MyDrive/SadTalker/inference.py --driven_audio {audio_filename} --source_image {source_image_path} --result_dir {result_dir} --still --preprocess full --enhancer gfpgan --checkpoint_dir {checkpoint_dir} --batch_size 1

# ensuring we are getting the latest generated video
def get_latest_generated_video(result_dir):
    video_files = [os.path.join(result_dir, file) for file in os.listdir(result_dir) if file.endswith(".mp4")]
    if not video_files:
        return None
    latest_video = max(video_files, key=os.path.getmtime)
    return latest_video

generated_video = get_latest_generated_video(result_dir)
print(f"Generated Video: {generated_video}")

# Displaying the video in the notebook
if generated_video:
    from IPython.display import display, Video
    display(Video(generated_video, embed=True, width=600, height=400))
else:
    print("No video file generated.")

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00,  8.84it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 17.03it/s]
mel:: 100% 4977/4977 [00:00<00:00, 42245.13it/s]
audio2exp:: 100% 498/498 [00:01<00:00, 452.98it/s]
Face Renderer:: 100% 4977/4977 [02:39<00:00, 31.23it/s]
The generated video is named /content/drive/MyDrive/SadTalker/results/2024_05_21_11.32.39/obama##generated_audio.mp4
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
seamlessClone:: 100% 4977/4977 [04:45<00:00, 17.43it/s]
The generated video is named /content/drive/MyDrive/SadTalker/results/2024_05_21_11.32.39/obama##generated_audio_full.mp4
face enhancer....
Face Enhancer:: 100% 4977/4977 [13:44<00:00,  6.04it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (656, 716) to (656, 720) to ensure video